## Initialisation
Before running the initialisation cell below, you must upload your completed DRI Batch Metadata Template file in Excel format to storage available to the Notebook. You should also create an output folder to store the generated XML files.

When you have completed this you should click on the run icon for the cell below which will read in your metadta file and set up some data that will be used in the rest of the Notebook.

Bear in mind that it is important to run the cells in Jupyter Notebooks in order. If you try to run the checking or processing cells before you have run this initialisation cell, you will get errors or unexpected results.

In [2]:
import pandas as pd
from os import path
import re

# Setup lists of valid fields which we will use for checking our metadata later
dc_fields = [
    'dc:identifier',
    'dc:title',
    'dcterms:alternative',
    'dc:creator',
    'dc:date',
    'dcterms:created',
    'dcterms:issued',
    'dc:description',
    'dc:rights',
    'dc:type',
    'dcterms:accessRights',
    'dc:language',
    'dc:contributor',
    'dc:source',
    'dc:coverage',
    'dcterms:spatial',
    'dcterms:temporal',
    'dc:subject',
    'dcterms:depicted',
    'dc:relation',
    'determs:isVersionOf',
    'dcterms:hasVersion',
    'dcterms:isPartOf',
    'dcterms:hasPart',
    'dcterms:isReferencedBy',
    'dcterms:references',
    'dcterms:isFormatOf',
    'dcterms:hasFormat']

marcrel_fields = ["abr","act","adp","rcp","anl","anm","ann","anc","apl","ape",
                  "app","arc","arr","acp","adi","art","ill","ard","asg","asn",
                  "fmo","att","auc","aue","aup","aut","aqt","aud","ato","ant",
                  "bnd","bdd","blw","bka","bkd","bkp","bjd","bpd","bsl","brl",
                  "brd","cll","cop","ctg","cas","cad","cns","chr","cng","cli",
                  "cor","col","clt","clr","cmm","cwt","com","cpl","cpt","cpe",
                  "cmp","cmt","ccp","cnd","con","csl","csp","cos","cot","coe",
                  "cts","ctt","cte","ctr","ctb","cpc","cph","crr","crp","cst",
                  "cou","crt","cov","cre","cur","dnc","dtc","dtm","dte","dto",
                  "dfd","dft","dfe","dgc","dgg","dgs","dln","dpc","dpt","dsr",
                  "drt","dis","dbp","dst","djo","dnr","drm","dbd","dub","edt",
                  "edc","edm","edd","elg","elt","enj","eng","egr","etr","evp",
                  "exp","fac","fld","fmd","fds","flm","fmp","fmk","fpy","frg",
                  "fmo","fon","fnd","gdv","gis","hnr","hst","his","ilu","ill",
                  "ink","ins","itr","ive","ivr","inv","isb","jud","jug","lbr",
                  "ldr","lsa","led","len","ltr","lil","lit","lie","lel","let",
                  "lee","lbt","lse","lso","lgd","ltg","lyr","mka","mfp","mfr",
                  "mrb","mrk","med","mdc","mte","mtk","mxe","mod","mon","mcp",
                  "mup","msd","mus","nrt","nan","onp","osp","opn","orm","org",
                  "oth","own","pan","ppm","pta","pth","pat","pnc","prf","prf",
                  "pma","pht","pad","ptf","ptt","pte","plt","pra","pre","prt",
                  "pop","prm","prc","pro","prn","prs","pmn","prd","prp","prg",
                  "pdr","pfr","crr","prv","pbl","pup","pbl","pbd","ppt","rdd",
                  "rpc","rap","rce","rcd","red","rxa","ren","rpt","rps","rth",
                  "rtm","res","rsp","rst","rse","rpy","rsg","rsr","rev","rbr",
                  "sce","sad","aus","scr","fac","scl","spy","sec","sll","std",
                  "stg","sgn","ins","sng","swd","sds","sde","spk","sfx","spn",
                  "sgd","stm","stn","str","stl","sht","srv","tch","tad","tcd",
                  "tld","tlg","tlh","tlp","tau","ths","trc","fac","trl","tyd",
                  "tyg","bkd","uvp","vdg","vfx","vac","wit","wde","wdc","wam",
                  "wac","wal","wat","waw","wfs","wfw","wft","win","wpr","wst",
                  "wts"]

xml_header = '<qualifieddc xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:marcrel="http://www.loc.gov/marc.relators/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.loc.gov/marc.relators/ http://imlsdcc2.grainger.illinois.edu/registry/marcrel.xsd" xsi:noNamespaceSchemaLocation="http://dublincore.org/schemas/xmls/qdc/2008/02/11/qualifieddc.xsd">'
xml_footer = '</qualifieddc>'

# Get the filename to read and the output folder
filename = ""
while True:
  if not path.isfile(filename):
    print('You must enter a valid filename for the file containing your metadata.')
  else:
    print(f'Successfully found the input file "{filename}"')
    break
  filename = input('Please enter the name of the file you have uploaded containing the cleaned metadata: ')

outputdir = ""
while True:
  if not path.isdir(outputdir):
    print('You must enter a valid folder name for the output folder, make sure you have created the folder first.')
  else:
    print(f'Successfully found the output folder "{outputdir}"')
    break
  outputdir = input('Please enter the name of the output folder: ')

print('If no errors are displayed here, you may now progress to execute the next code step.')
print('You may run the proceeding code steps as many times as you want without having to re-run this Initialisation step.')
print('You only need to re-run the Initialisation step if you see an error here, or if you want to change the input file or output folder.')
    
def lookup_case_insensitive(value, choices):
    """Return the matching value from choices with original case, or None if not found."""
    value_cf = value.casefold()
    lookup = {c.casefold(): c for c in choices}
    return lookup.get(value_cf)

class CaseInsensitiveLookupManager:
    def __init__(self):
        self.lookups = {}

    def register(self, name, choices):
        """Register a lookup set by name."""
        self.lookups[name] = {c.casefold(): c for c in choices}

    def get(self, name, value):
        """Case-insensitive lookup in the named set."""
        return self.lookups.get(name, {}).get(value.casefold())

    def find_in(self, names, value):
        """
        Search for value in multiple sets in order.
        Returns (set_name, match) or (None, None) if not found.
        """
        value_cf = value.casefold()
        for name in names:
            match = self.lookups.get(name, {}).get(value_cf)
            if match is not None:
                return name, match
        return None, None

You must enter a valid filename for the file containing your metadata.


Please enter the name of the file you have uploaded containing the cleaned metadata:  DRI_Metadata Template.xlsx


Successfully found the input file "DRI_Metadata Template.xlsx"
You must enter a valid folder name for the output folder, make sure you have created the folder first.


Please enter the name of the output folder:  out


Successfully found the output folder "out"
If no errors are displayed here, you may now progress to execute the next code step.
You may run the proceeding code steps as many times as you want without having to re-run this Initialisation step.
You only need to re-run the Initialisation step if you got an error here, or if you want to change the input file or output folder


## Check and Clean the Metadata
The following cell will perform some checks on your metadata to make sure that it can be converted to Dublin Core XML files.

1. First it will discard any empty columns, necessary as there will likely be columns in the metadata template that are not relevant for your data. We don't want to include these as empty Dublin Core elements in our xml files.

2. Next it will check that all of the column headers match a valid Dublin Core element. You may have added additional columns to handle multiple values, e.g. for multiple Subjects you may have dc:Subject, dc:Subject2, dc:Subject3, etc. The Notebook will ask you to clarify the metadata mapping for any columns where this is unclear.

In [ ]:
# Read in the metadata file
df = pd.read_excel(filename, 1)

# drop any unused columns
df = df.dropna(axis=1, how='all')

drop_fields = []
mappings = {}

marcrel_pattern = re.compile('^marcrel:.*', re.IGNORECASE)

lookup_manager = CaseInsensitiveLookupManager()
lookup_manager.register("dc", dc_fields)
lookup_manager.register("marcrel", marcrel_fields)

for field in df.columns:
    field_cf = field.casefold()

    if field_cf == "filename":
        continue

    if marcrel_pattern.match(field):
        ns, code = field.split(":")
        _, match = lookup_manager.find_in(["marcrel"], code)
        if not match:
            print(f'Field "{field}" is not a valid MARC Relator code.')
            tmp = input(
                f'Please specify the field (in the format marcrel:xxx) for mapping. '
                f'Hit enter to ignore this column: '
            )
            if tmp == "":
                print(f'Dropping column {field}')
                drop_fields.append(field)
            else:
                mappings[field] = tmp
        else:
            mappings[field] = "marcrel:"+match

    else:
        _, match = lookup_manager.find_in(["dc"], field)
        if not match:
            print(f'Field "{field}" is not a valid Dublin Core field.')
            tmp = input(
                f'Please specify the dc field (in the format dc:xxxxxx or dcterms:xxxxxx) for mapping. '
                f'Hit enter to ignore this column: '
            )
            if tmp == "":
                print(f'Dropping column {field}')
                drop_fields.append(field)
            else:
                mappings[field] = tmp
        else:
            mappings[field] = match

df = df.drop(drop_fields, axis=1)

Field "dc:Subject 2" is not a valid Dublin Core field.
Please specify the dc field (in the format dc:xxxxxx or dcterms:xxxxxx) for mapping. Hit enter to ignore this column: dc:subject
Field "dc:Subject 3" is not a valid Dublin Core field.
Please specify the dc field (in the format dc:xxxxxx or dcterms:xxxxxx) for mapping. Hit enter to ignore this column: dc:subject
Field "marcrel:***" is not a valid MARC Relator code.
Please specify the field (in the format marcrel:xxx) for mapping. Hit enter to ignore this column: 
Dropping column marcrel:***


## Process Metadata and create XML files
This next cell will iterate through all rows in your metadata spreadsheet, converting the cell values to xml and write these out to the output file which will be named for your Filename cell. A zip file metadata.zip, and a tar archive metadata.tar.gz, containing of your output folder will also be created. These can then be downloaded to your local machine and extracted, before ingesting into DRI or another Repository or application that supports Dublin Core metadata.


In [ ]:
for index, row in df.iterrows():
  xmlfile = open(f'{outputdir}/{row["Filename"]}.xml', 'w')
  xmlfile.write(xml_header)
  for field in df.columns:
    if field == "Filename":
      continue
    elif pd.isna(row[field]):
      continue
    else:
      tmp = f'<{mappings[field]}>{row[field]}</{mappings[field]}>\n'
      xmlfile.write(tmp)
  xmlfile.write(xml_footer)
  xmlfile.close()

!zip -r metadata.zip $outputdir
!tar -czf metadata.tar.gz $outputdir


  adding: out// (stored 0%)
  adding: out//file2.xml (deflated 54%)
  adding: out//file1.xml (deflated 54%)
